# VGG16 TESTING

In [1]:
import torchvision.models as models
from time import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from data import input_data

In [2]:
vgg16 = models.vgg16(pretrained="imagenet")
device = torch.device("cuda:1")

In [3]:
vgg16.classifier = nn.Sequential(nn.Linear(25088,4096),
                                nn.ReLU6(True),
                                nn.Dropout(0.5,inplace=False),
                                nn.Linear(4096,4096),
                                nn.ReLU6(True),
                                nn.Dropout(0.5,inplace=False),
                                nn.Linear(4096,10))
vgg16.to(device)
print("")

In [4]:
optimizer = optim.Adam(vgg16.parameters(), lr=0.0001)
criterion_d = nn.CrossEntropyLoss()
input_train = input_data(root_dir = "/home/hemant/net/easy_net/data/train/", type = "valid")
train_dl =  DataLoader(input_train, batch_size=32,shuffle=True, num_workers=4)
input_valid = input_data(root_dir = "/home/hemant/net/easy_net/data/test/", type = "valid")
valid_dl =  DataLoader(input_valid, batch_size=64,shuffle=False, num_workers=4)

total number of classes in valid are :  10
size of data 9984
total number of classes in valid are :  10
size of data 996


# Training

In [ ]:
vgg16.load_state_dict(torch.load("../weights/vgg16.pth"))

In [15]:
for j in range(2):
    print("start of epoch: ", j+1)
    #Training
    running_loss, switch_ = 0, 0
    start = time()
    for i, data in enumerate(train_dl, 0):

        input, target, img_name, number_of_class = data
        input, target = (input.type(torch.float32)).to(device), target.to(device)

        out = vgg16(input)

        loss = criterion_d(out, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()


        optimizer.zero_grad()
        
        # print every 25 mini-batches
        if i % 25 == 24:
            print('[%d, %5d] loss: %.3f' %(j + 1, i + 1, running_loss))
            running_loss = 0
    end = time()
    print(" It took : ", (end - start)/60, " mins for the last training epoch")

start of epoch:  1
[1,    25] loss: 0.411
[1,    50] loss: 2.144
[1,    75] loss: 2.354
[1,   100] loss: 0.836
[1,   125] loss: 1.486
[1,   150] loss: 0.979
[1,   175] loss: 0.927
[1,   200] loss: 1.413
[1,   225] loss: 2.178
[1,   250] loss: 1.514
[1,   275] loss: 2.735
[1,   300] loss: 1.372
 It took :  7.348183190822601  mins for the last training epoch
start of epoch:  2
[2,    25] loss: 1.001
[2,    50] loss: 2.278
[2,    75] loss: 0.883
[2,   100] loss: 0.632
[2,   125] loss: 0.960
[2,   150] loss: 1.398
[2,   175] loss: 0.543
[2,   200] loss: 0.613
[2,   225] loss: 1.008
[2,   250] loss: 0.867
[2,   275] loss: 1.079
[2,   300] loss: 0.519
 It took :  6.559609619776408  mins for the last training epoch


# Testing

In [16]:
torch.save(vgg16.state_dict(),"../weights/" + "vgg16" + ".pth")

In [17]:
running_loss, acc, num, length = 0, 0, 0, 0
with torch.no_grad():
    start = time()
    for i, data in enumerate(valid_dl, 0):
        vgg16.eval()

        input, target, img_name, number_of_class = data
        input, target = (input.type(torch.float32)).to(device), target.to(device)

        out = vgg16(input)

        loss = criterion_d(out, target)

        out , predicted = torch.max(out, 1)
        for k in range(len(target)):
            if target[k] == predicted[k].item():
                num = num + 1
        length = length + len(target)
    acc = (num/length)*100
    end = time()
    print("accuracy and val loss is : ",acc,",",running_loss/(i+1), " --AND-- ", " It took : ", (end - start), " seconds ")

accuracy and val loss is :  85.34136546184739 , 0.0  --AND--   It took :  11.654378414154053  seconds 
